In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".10"

In [2]:
import pandas as pd
from model import Brain
from submodels import factories
import matplotlib.pyplot as plt
import pandas as pd
from itertools import accumulate
import numpy as np
from collections import defaultdict
import re

from lib.sde.grn2 import GRNMain2
from lib.sde.mutate import mutate_grn2
from lib.ga.utils import weighted_selection
from lib.ga.objective import Objective
from jf.profiler import Profiler
from jf.utils.export import Exporter
from jf.autocompute.jf import O
from jf.models.stringmodel import read_model

from lib.sde.gene3 import ParamGene3
from collections import Counter

In [3]:
sm = read_model("generation")

In [4]:
class Solution:
    def __init__(self, grn):
        self.grn = grn
        
    def copy(self):
        return Solution(self.grn.copy())
        
    def mutate(self):
        mutate_grn2(self.grn)

In [5]:
def get_bb(grn):
    ccls = factories["grn3"](grn=grn)
    bb = Brain(time_step=0.5, verbose=False, start_population=5, max_pop_size=1e3,
            cell_cls=ccls, end_time=70, start_time=50, silent=True, record_population=True)
    return bb

In [6]:
def run_grn(prun, grn):
    get_bb(prun, grn)
    bb.run()
    return bb

In [7]:
exporter = Exporter(name="objective_0.25_g7_v5")

Exporting at output/objective_0.25_g7_v5


In [8]:
all_generations = {int(sm.extract(x)["generation"]): exporter.load(x) for x in list(filter(sm.match, exporter.list()))}

In [16]:
def get_diff_grn(grn_a, grn_b):
    return list(zip(*np.where(~np.isclose(grn_a._params, grn_b._params))))

In [17]:
grn_a = all_generations[5]["solution"][0].grn
grn_b = all_generations[5]["solution"][4].grn

def get_diff_grn_trees(grn_a, grn_b):
    return [i for i, (t_a, t_b) in enumerate(zip(grn_a.genes, grn_b.genes)) if repr(t_a.tree) != repr(t_b.tree)]

get_diff_grn_trees(grn_a, grn_b)

[3, 6]

In [91]:
# fitness match
def one_delta_fitness(all_generations, generation, min_diff=1., only_pos=False, only_neg=False):
    grns_begin = all_generations[generation]["solution"]
    fitness_begin = all_generations[generation + 1]["fitness"]
    transition = all_generations[generation + 1]["transition"]
    grns_end = all_generations[generation + 1]["solution"]
    fitness_end = all_generations[generation + 2]["fitness"]

    # matching
    ls_matches = []
    for i, prev_id in enumerate(transition):
        if prev_id == -1:
            continue
        grn_a = grns_begin[prev_id].grn
        grn_b = grns_end[i].grn
        delta_fitness = fitness_end[i] - fitness_begin[prev_id]
        diff = get_diff_grn(grn_a, grn_b)
        tree_diff = get_diff_grn_trees(grn_a, grn_b) 
        for x in diff:
            if abs(delta_fitness) < min_diff:
                continue
            if only_pos and delta_fitness < 0:
                continue
            if only_neg and delta_fitness > 0:
                continue
            ls_matches.append((
                grn_a._params[x], grn_b._params[x], ParamGene3[x[0]], x[1], delta_fitness
            ))
            
        for x in tree_diff:
            if abs(delta_fitness) < min_diff:
                continue
            if only_pos and delta_fitness < 0:
                continue
            if only_neg and delta_fitness > 0:
                continue
            ls_matches.append((
                grn_a.genes[x].tree, grn_b.genes[x].tree, "tree", x, delta_fitness
            ))
    
    return ls_matches

In [21]:
def print_matches(matches):
    for match in matches:
        if match[2] == "tree":
            print("{} -> {} for {}_{} : {:.3f}".format(*match))
        else:
            print("{:.3f} -> {:.3f} for {}_{} : {:.3f}".format(*match))

In [ ]:
grns_begin = all_generations[generation]["solution"]
fitness_begin = all_generations[generation + 1]["fitness"]
transition = all_generations[generation + 1]["transition"]
grns_end = all_generations[generation + 1]["solution"]
fitness_end = all_generations[generation + 2]["fitness"]

In [32]:
def get_fitness_at(all_generations, generation, index):
    """Generation of the solution, therefore corrected"""
    return all_generations[generation + 1]["fitness"][index]

def get_transition(all_generations, from_, to_, index):
    assert from_ - 1 == to_
    return all_generations[from_]["transition"][index]

def get_grn_at(all_generations, generation, index):
    return all_generations[generation]["solution"][index].grn

In [37]:
max_gen = 10
fit = all_generations[max_gen]["fitness"]
idx = fit.index(max(fit))
sols = [(all_generations[max_gen - 1]["solution"][idx], 0)]
for i in range(1, max_gen - 1):
    trans = all_generations[max_gen - i]["transition"]
    idx = trans[idx]
    new_fit = all_generations[max_gen - i]["fitness"][idx]
    delta_fit = fit - new_fit
    fit = new_fit
    if idx == -1:
        print("-1 found")
        break
    sols.append((all_generations[max_gen - i - 1]["solution"][idx].grn, delta_fit))
sols = sols[::-1]

In [47]:
max_gen = 70
fit = all_generations[max_gen + 1]["fitness"]
idx = fit.index(max(fit))
sols = []

for i in range(max_gen, 1, -1):
    grn = get_grn_at(all_generations, i, idx)
    trans = get_transition(all_generations, i, i - 1, idx)
    if trans == -1:
        print("-1 found")
        break
    new_fit = get_fitness_at(all_generations, i - 1, trans)
    sols.append((grn, fit - new_fit, fit))
    fit = new_fit
    idx = trans
sols = sols[::-1]

In [48]:
def get_summary_transition(grn_a, grn_b, delta_fit=0):
    ls_matches = []
    diff = get_diff_grn(grn_a, grn_b)
    tree_diff = get_diff_grn_trees(grn_a, grn_b) 
    for x in diff:
        ls_matches.append((
            grn_a._params[x], grn_b._params[x], ParamGene3[x[0]], x[1], delta_fit
        ))

    for x in tree_diff:
        ls_matches.append((
            grn_a.genes[x].tree, grn_b.genes[x].tree, "tree", x, delta_fit
        ))
    return ls_matches

In [49]:
for i in range(len(sols) - 2):
    print(f"\tT{i} -> {i+1} : {sols[i][2]}")
    print_matches(get_summary_transition(sols[i][0], sols[i+1][0], sols[i+1][1]))

	T0 -> 1 : 1.5520041719722122
6.242 -> 7.416 for deg_1 : -0.439
	T1 -> 2 : 1.112908681982816
(((5 AND 6) OR 4) AND 1) -> ((6 OR 4) AND 1) for tree_0 : 0.776
	T2 -> 3 : 1.8885931230636694
7.833 -> 8.113 for noise_4 : -0.262
	T3 -> 4 : 1.6264379421915813
0.162 -> 0.764 for init_6 : -0.323
8.925 -> 5.702 for expr_1 : -0.323
4.760 -> 7.515 for expr_5 : -0.323
	T4 -> 5 : 1.3034632220822766
((1 AND 0) AND 3) -> ((1 AND 5) AND 3) for tree_6 : 0.379
	T5 -> 6 : 1.6820288169020392
4.355 -> 4.264 for noise_6 : 0.455
(NOT (0 OR 1) OR 5) -> NOT (0 OR 1) for tree_2 : 0.455
NOT (NOT 2 OR (5 AND 0)) -> NOT (NOT 2 OR (5 AND (0 OR 4))) for tree_5 : 0.455
	T6 -> 7 : 2.1374808348171914
5 -> (5 OR 2) for tree_4 : -0.368
	T7 -> 8 : 1.7694827623049372
5.702 -> 7.482 for expr_1 : -0.301
	T8 -> 9 : 1.4683052762540127
7.260 -> 3.359 for noise_0 : 1.394
9.708 -> 5.776 for b_3 : 1.394
7.416 -> 1.108 for deg_1 : 1.394
NOT (0 OR 1) -> NOT 1 for tree_2 : 1.394
(NOT NOT 1 AND (2 AND NOT 6)) -> (NOT NOT 4 AND (2 AND N

In [45]:
sols[8]

(>> G_0: init: 0.52; noise: 7.26; b: 3.62; m: 1.17; expr: 9.09; deg: 0.16; theta: 1.00; tree : ((((5 AND 6) AND 3) OR 4) AND 1)
 >> G_1: init: 0.50; noise: 4.34; b: 2.45; m: 5.39; expr: 8.92; deg: 3.29; theta: 4.61; tree : 1
 >> G_2: init: 2.19; noise: 7.63; b: 4.62; m: 7.76; expr: 0.63; deg: 3.85; theta: 7.95; tree : ((NOT 1 AND 0) OR 5)
 >> G_3: init: 0.02; noise: 8.18; b: 9.71; m: 4.60; expr: 9.20; deg: 7.70; theta: 2.38; tree : (NOT NOT 1 AND NOT 4)
 >> G_4: init: 0.44; noise: 2.42; b: 3.76; m: 2.77; expr: 1.72; deg: 3.71; theta: 5.17; tree : 5
 >> G_5: init: 0.67; noise: 3.06; b: 5.08; m: 6.41; expr: 4.76; deg: 8.31; theta: 4.96; tree : ((NOT 2 OR 6) OR ((5 AND (1 OR 4)) OR 3))
 >> G_6: init: 0.16; noise: 4.35; b: 8.11; m: 6.89; expr: 3.31; deg: 7.83; theta: 0.71; tree : ((1 AND 2) AND 6),
 array([-1.65117237, -0.13904795, -0.30589578, -0.35227673, -1.63407464,
        -0.05234554, -1.65083686, -1.18115761, -0.69047206, -0.05769425,
        -1.22273442, -0.07353205, -0.2640596 , -

In [18]:
get_diff_grn(sols[0], sols[1])

[(2, 6)]

In [15]:
sols[1]

>> G_0: init: 0.52; noise: 7.26; b: 9.00; m: 1.17; expr: 9.09; deg: 0.16; theta: 1.00; tree : (((5 AND 6) OR 4) AND 1)
>> G_1: init: 0.50; noise: 4.34; b: 2.45; m: 5.39; expr: 8.92; deg: 6.24; theta: 4.61; tree : 1
>> G_2: init: 2.19; noise: 7.63; b: 4.62; m: 7.76; expr: 0.63; deg: 3.85; theta: 7.95; tree : (NOT (0 OR 1) OR 5)
>> G_3: init: 0.82; noise: 8.18; b: 9.71; m: 4.60; expr: 9.20; deg: 7.70; theta: 2.38; tree : (NOT NOT 1 AND (2 AND NOT 6))
>> G_4: init: 0.44; noise: 7.83; b: 3.76; m: 3.64; expr: 1.72; deg: 3.71; theta: 4.74; tree : 5
>> G_5: init: 0.67; noise: 3.06; b: 5.08; m: 6.41; expr: 4.76; deg: 8.31; theta: 4.96; tree : NOT (NOT 2 OR (5 AND 0))
>> G_6: init: 0.16; noise: 4.35; b: 5.71; m: 6.89; expr: 3.31; deg: 7.83; theta: 0.71; tree : ((1 AND 0) AND 3)

In [19]:
sols[0]

>> G_0: init: 0.52; noise: 7.26; b: 9.00; m: 1.17; expr: 9.09; deg: 0.16; theta: 1.00; tree : (((5 AND 6) OR 4) AND 1)
>> G_1: init: 0.50; noise: 4.34; b: 2.45; m: 5.39; expr: 8.92; deg: 6.24; theta: 4.61; tree : 1
>> G_2: init: 2.19; noise: 7.63; b: 4.62; m: 7.76; expr: 0.63; deg: 3.85; theta: 7.95; tree : (NOT (0 OR 1) OR 5)
>> G_3: init: 0.82; noise: 8.18; b: 9.71; m: 4.60; expr: 9.20; deg: 7.70; theta: 2.38; tree : (NOT NOT 1 AND (2 AND NOT 6))
>> G_4: init: 0.44; noise: 7.83; b: 3.76; m: 3.64; expr: 1.72; deg: 3.71; theta: 4.74; tree : 5
>> G_5: init: 0.67; noise: 3.06; b: 5.08; m: 6.41; expr: 4.76; deg: 8.31; theta: 4.96; tree : NOT (NOT 2 OR (5 AND 0))
>> G_6: init: 0.16; noise: 4.35; b: 8.11; m: 6.89; expr: 3.31; deg: 7.83; theta: 0.71; tree : ((1 AND 2) AND 3)